# Binary Image Classification
The following notebook is responsible for demonstrating how one would go about creating and evaluating a model using the two image generator. Namely, 

In [1]:
import tensorflow as tf
import scipy
import numpy as np
import pandas as pd
import keras

from matplotlib import pyplot as plt
from tensorflow.keras import datasets, layers, models
from mpl_toolkits.axes_grid1 import ImageGrid

from preprocessing.data_generator import CustomGen
from meta.paths import PATH_TO_TRAIN_PAIRS, PATH_TO_TEST_PAIRS
from models.prototype import create_model

In [3]:
BATCH_SIZE = 32
IMAGE_SIZE = (100, 100)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
EPOCHS = 1
FIT_AND_SAVE = False

# Create and Traing Model

In [4]:
if FIT_AND_SAVE: # creates a model from scratch and saves it
    train_pairs_df = pd.read_csv(PATH_TO_TRAIN_PAIRS) 
    train_generator = CustomGen(train_pairs_df,
                                shuffle=True, 
                                batch_size=BATCH_SIZE, 
                                image_size=IMAGE_SIZE)
    model = create_model(IMAGE_SHAPE)
    history = model.fit(train_generator, 
                        epochs=EPOCHS, 
                        batch_size=BATCH_SIZE)
    model.save("model.h5")
else:  # loads previous run
    model = keras.models.load_model("model.h5")

Found 119502 validated image filenames belonging to 2 classes.
Found 119502 validated image filenames belonging to 2 classes.


# Test Model

In [5]:
test_pairs = pd.read_csv(PATH_TO_TEST_PAIRS)
test_generator = CustomGen(test_pairs, shuffle=True, batch_size=BATCH_SIZE, image_size=IMAGE_SIZE)
predictions = model.predict(test_generator)

Found 38627 validated image filenames belonging to 2 classes.
Found 38627 validated image filenames belonging to 2 classes.


In [25]:
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, auc

pred_classifications = list(map(lambda s: 0 if s < .5 else 1, predictions))
y_true = test_pairs["class_label"]

ap_score = average_precision_score(y_true, predictions)
auc_score = auc(y_true, predictions)
a_score = accuracy_score(y_true, pred_classifications)

print("Test Accuracy: \t", a_score)
print("Test AP: \t", ap_score)
print("Test AUC: \t", auc_score)

Test Accuracy: 	 0.46672016982939396
Test AP: 	 0.2968365390470042
Test AUC: 	 0.786208987236023
